<a href="https://colab.research.google.com/github/uni2237/Data_Analysis/blob/master/%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 분석 환경 설정 및 데이터 로딩

In [207]:
# 모델 구현 시간 체크
import time
# 시작시간 체크
start = time.time()

# 패키지 로딩
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글폰트 적용
plt.rcParams['font.family'] = 'Malgun Gothic'

# 수입 데이터 로딩 (가상데이터)
df_org = pd.read_csv('train.csv')
df_org_ts=pd.read_csv('test.csv')

# 데이터 전처리

### 2.1 전처리 기본

- 원산지 적출 컬럼생성

In [208]:

df_org["원산지적출"] = df_org["원산지국가코드"] +df_org["적출국가코드"]
df_org_ts["원산지적출"] = df_org_ts["원산지국가코드"] +df_org_ts["적출국가코드"]

- 원산지/ 적출/ 원산지적출/ 신고인부호 리스트 생성

In [209]:
# 원산지 / 적출/ 원산지적출 리스트 생성
def make_list(df_org):
  result_1=df_org['원산지국가코드'][df_org['우범여부']==1].value_counts()
  result_0=df_org['원산지국가코드'][df_org['우범여부']==0].value_counts()

  result_11=df_org['적출국가코드'][df_org['우범여부']==1].value_counts()
  result_01=df_org['적출국가코드'][df_org['우범여부']==0].value_counts()


  result_all = df_org['원산지국가코드'].value_counts()
  result_all1 = df_org['적출국가코드'].value_counts()

  
  result_12=df_org['원산지적출'][df_org['우범여부']==1].value_counts()
  result_02=df_org['원산지적출'][df_org['우범여부']==0].value_counts()
  result_all2 = df_org['원산지적출'].value_counts()

  a_b=[]
  a=[]
  b=[]
  for i in result_1.keys():
    #print(i,"의 우범비율 %.2f%%" % (result_1[i] / result_all[i]* 100.0), "   (",result_all[i],"개 중 ",result_1[i],")")
    if int(result_1[i] / result_all[i]* 100.0)>=40:
      a.append(i)

  for i in result_11.keys():
      #print(i,"의 우범비율 %.2f%%" % (result_11[i] / result_all1[i]* 100.0), "   (",result_all1[i],"개 중 ",result_11[i],")")
      if int(result_11[i] / result_all1[i]* 100.0)>=40:
        b.append(i)
  for i in sorted(result_12.keys()):
      #print(i,"의 우범비율 %.2f%%" % (result_12[i] / result_all2[i]* 100.0), "   (",result_all2[i],"개 중 ",result_12[i],")")
      if int(result_12[i] / result_all2[i]* 100.0)>=70:
        a_b.append(i)
  print("a",a)
  print("b",b)
  print("a_b",a_b)
  print(len(a_b))

  #신고인 부호 리스트 생성
  big_list=[]  # 1로 바꿔줄 신고인부호 저장할 리스트 (우범 횟수가 100이상 또는 우범 비열이 50% 이상인 신고인들을 1로 둠)
  result_1=df_org['신고인부호'][df_org['우범여부']==1].value_counts()
  result_all=df_org['신고인부호'].value_counts()
  count=0
  for i in result_1:
    if i>=100:
      count+=1
    else:
      break

  for i in result_1.keys()[0:count]:
      big_list.append(i)

  for i in sorted(result_1.keys()):
      if result_1[i] / result_all[i]* 100.00 >=50:
          big_list.append(i)

  return a,b,a_b,big_list

- 원산지/ 적출/ 원산지적출/ 신고인부호 리스트 저장

In [210]:
a,b,a_b,big_list=make_list(df_org)

a ['KG', 'NO', 'BA', 'SM', 'UA', 'HR', 'BO', 'AM', 'GT']
b ['NO', 'UG', 'KG', 'AR', 'IE', 'HR', 'LI']
a_b ['AMIT', 'ARAR', 'ATTW', 'BADE', 'CNAT', 'CNTR', 'ESSG', 'GBJP', 'GBNL', 'GTNL', 'HRVN', 'INMY', 'JPPL', 'KRMX', 'PTDE', 'THGB', 'THVN', 'TRNL']
18


- 전체 전처리 함수

In [211]:
def data_process(df_org):

  

  # One-Hot-Encoding (3, 12, 15, 18, 19)
  #df_org = pd.get_dummies(df_org, columns = ['통관지세관부호', '징수형태코드', '운송수단유형코드', '적출국가코드', '원산지국가코드'])
  df_org = pd.get_dummies(df_org, columns = ['통관지세관부호', '징수형태코드', '운송수단유형코드'])

  
  # 원산지 / 적출 / 원산지 적출

  def function_a(row):
      if row['원산지국가코드'] in a:
          return 1
      else:
          return 0
  def function_b(row):
    if row['적출국가코드'] in b:
        return 1
    else:
        return 0
  def function_a_b(row):
    if row['원산지적출'] in a_b:
        return 1
    else:
        return 0
  df_org['원산지국가코드'] =df_org.apply(function_a,axis=1)
  df_org['적출국가코드'] =df_org.apply(function_b,axis=1)
  df_org['원산지적출'] =df_org.apply(function_a_b,axis=1)


  # 수치형 변수를 자연로그로 변환(13, 14)
  for var in ['신고중량(KG)', '과세가격원화금액']:
      df_org[var] = df_org[var].apply(lambda x: np.log1p(x))

    
  # 신고 날짜 전처리를 위해 함수 정의
  def extract_month(row): return row.split('-')[1]
  def extract_day(row) : return int(row.split('-')[2])//10


  # 2.신고 일자
  df_org['월'] = df_org['신고일자'].apply(extract_month)
  df_org['일'] = df_org['신고일자'].apply(extract_day)
  del df_org['신고일자']
  df_org = pd.get_dummies(df_org, columns = ['월', '일'])


  # 4.신고인부호
  def function_man(row):
      if row['신고인부호'] in big_list:
          return 1
      else:
          return 0
  df_org['신고인부호'] =df_org.apply(function_man,axis=1)


  # 6.해외거래처부호
  df = df_org['해외거래처부호'].fillna('missing') # nan을 missing으로 치환
  df_org['해외거래처부호'] = df 
  df_org=pd.get_dummies(df_org, columns = ['해외거래처부호'])
  missing = df_org.filter(regex='^해외거래처부호_missing') 
  df_org.drop(df_org.filter(regex='^해외거래처부호_'),axis=1,inplace=True) #missing 컬럼 제외하고 다지우기
  df_org['해외거래처부호_missing'] =  missing


  # 7.특송업체번호
  df = df_org['특송업체부호'].fillna('missing') # nan을 missing으로 치환
  df_org['특송업체부호'] = df 
  df_org=pd.get_dummies(df_org, columns = ['특송업체부호'])
  missing = df_org.filter(regex='^특송업체부호_missing') 
  df_org.drop(df_org.filter(regex='^특송업체부호_'),axis=1,inplace=True) #missing 컬럼 제외하고 다지우기
  df_org['특송업체부호_missing'] =  missing


  # 8.수입통관계획코드
  df_org=pd.get_dummies(df_org, columns = ['수입통관계획코드'])
  df_org['수입통관계획코드AG'] = 0
  df_org["수입통관계획코드CD"] = df_org["수입통관계획코드_C"] +df_org["수입통관계획코드_D"]
  df_org["수입통관계획코드EF"] = df_org["수입통관계획코드_E"] +df_org["수입통관계획코드_F"]
  df_org["수입통관계획코드HZ"] = df_org["수입통관계획코드_H"] +df_org["수입통관계획코드_Z"]
  df_org.drop(df_org.filter(regex='^수입통관계획코드_'),axis=1,inplace=True)


  # 9.수입신고구분코드
  df_org=pd.get_dummies(df_org, columns = ['수입신고구분코드'])
  df_org['수입신고구분코드CFG'] = 0


  # 10.수입거래구분코드
  df_org['수입거래구분코드_십'] = df_org['수입거래구분코드'] // 10
  df_org['수입거래구분코드_일'] = df_org['수입거래구분코드'] % 10
  del df_org['수입거래구분코드']
  df_org = pd.get_dummies(df_org, columns = ['수입거래구분코드_십', '수입거래구분코드_일'])


  # 11.수입종류코드(
  df_org.loc[df_org['수입종류코드'] == 21, '수입종류코드'] = 0
  df_org.loc[df_org['수입종류코드'] == 11, '수입종류코드'] = 0
  df_org.loc[df_org['수입종류코드'] != 0, '수입종류코드'] = 1
  df_org = pd.get_dummies(df_org, columns = ['수입종류코드'])


  # 16.반입보세구역부호
  def extract_1(row): return str(row)[0]
  def extract_2(row): return str(row)[1]
  def extract_3(row): return str(row)[0]
  def extract_4(row): return str(row)[1]
  df_org['반입보세구역부호_1'] = df_org['반입보세구역부호'].apply(extract_1)
  df_org['반입보세구역부호_2'] = df_org['반입보세구역부호'].apply(extract_2)
  df_org['반입보세구역부호_3'] = df_org['반입보세구역부호'].apply(extract_3)
  df_org['반입보세구역부호_4'] = df_org['반입보세구역부호'].apply(extract_4)
  df_org = pd.get_dummies(df_org, columns = ['반입보세구역부호_1', '반입보세구역부호_2', '반입보세구역부호_3', '반입보세구역부호_4'])
  del df_org['반입보세구역부호']


  # 17.HS10단위부호
  df_org['HS10단위부호_1'] = df_org['HS10단위부호'].apply(extract_1)
  df_org['HS10단위부호_2'] = df_org['HS10단위부호'].apply(extract_2)
  df_org = pd.get_dummies(df_org, columns = ['HS10단위부호_1', 'HS10단위부호_2'])
  del df_org['HS10단위부호']


  # 20.관세율 구분 코드
  df_org=pd.get_dummies(df_org, columns = ['관세율구분코드']) # 관세율구분코드를 원핫인코딩으로 변환
  df_org["관세율구분_A"]=df_org.iloc[:,-35]   # 각 범위로 그룹화,
  df_org["관세율구분_CE"]=df_org.iloc[:,-35:-30].sum(axis=1)
  df_org["관세율구분_FC"]=df_org.iloc[:,-31:-22].sum(axis=1)
  df_org["관세율구분_FE"]=df_org.iloc[:,-23:-9].sum(axis=1)
  df_org["관세율구분_LP"]=df_org.iloc[:,-10:-7].sum(axis=1)
  df_org["관세율구분_RW"]=df_org.iloc[:,-8:-5].sum(axis=1)
  df_org.drop(df_org.filter(regex='^관세율구분코드_'),axis=1,inplace=True)

  # 1 2 3 4 [5] 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 [21] 22 23

  

  del df_org['수입자부호']
  del df_org['관세율']
  del df_org['Unnamed: 0']
  return df_org

- 전처리 결과 저장 및 불필요한 변수 삭제 (trian.csv)

In [212]:

# 불필요한 변수 삭제(1, 22, 23)
del df_org['신고번호']
del df_org['검사결과코드']
del df_org['핵심적발']
df_org=data_process(df_org)

df_org_train = df_org

전처리 결과 저장 및 불필요한 변수 삭제 (test.csv)


In [213]:
df_org_ts=data_process(df_org_ts)
test_data = df_org_ts


print("train_data, test_data",df_org_train.shape, test_data.shape)

train_data, test_data (76837, 165) (23163, 159)


In [224]:
df=[]
for i in df_org_train.keys():
  df.append(i)
print(df)
df=[]
for i in test_data.keys():
  df.append(i)
print(df)

['신고인부호', '신고중량(KG)', '과세가격원화금액', '적출국가코드', '원산지국가코드', '원산지적출', '통관지세관부호_10', '통관지세관부호_12', '통관지세관부호_13', '통관지세관부호_14', '통관지세관부호_16', '통관지세관부호_17', '통관지세관부호_20', '통관지세관부호_21', '통관지세관부호_23', '통관지세관부호_30', '통관지세관부호_33', '통관지세관부호_39', '통관지세관부호_40', '통관지세관부호_41', '통관지세관부호_50', '통관지세관부호_53', '통관지세관부호_56', '통관지세관부호_60', '통관지세관부호_62', '통관지세관부호_70', '통관지세관부호_71', '통관지세관부호_80', '통관지세관부호_81', '통관지세관부호_82', '통관지세관부호_90', '통관지세관부호_100', '통관지세관부호_101', '통관지세관부호_102', '통관지세관부호_110', '통관지세관부호_120', '통관지세관부호_121', '통관지세관부호_122', '통관지세관부호_130', '통관지세관부호_131', '통관지세관부호_140', '통관지세관부호_150', '통관지세관부호_151', '통관지세관부호_152', '통관지세관부호_153', '통관지세관부호_154', '징수형태코드_0', '징수형태코드_11', '징수형태코드_12', '징수형태코드_13', '징수형태코드_14', '징수형태코드_18', '징수형태코드_21', '징수형태코드_33', '징수형태코드_43', '운송수단유형코드_10', '운송수단유형코드_20', '운송수단유형코드_30', '운송수단유형코드_40', '운송수단유형코드_50', '운송수단유형코드_90', '월_01', '월_02', '월_03', '월_04', '월_05', '월_06', '월_07', '월_08', '월_09', '일_0', '일_1', '일_2', '일_3', '해외거래처부호_missing', '특송업체부호_missing', '수입통관계획코드AG', '수입통

###2.2 훈련데이터와 테스트 데이터 분리

In [214]:
df_org_train = df_org.iloc[:int(len(df_org)*0.9), :]
df_org_test = df_org.iloc[int(len(df_org)*0.9):, :]

print(df_org_train.shape, df_org_test.shape)

(69153, 165) (7684, 165)


In [215]:
df_org_train.head(6)

,신고인부호,신고중량(KG),과세가격원화금액,적출국가코드,원산지국가코드,우범여부,원산지적출,통관지세관부호_10,통관지세관부호_12,통관지세관부호_13,통관지세관부호_14,통관지세관부호_16,통관지세관부호_17,통관지세관부호_20,통관지세관부호_21,통관지세관부호_23,통관지세관부호_30,통관지세관부호_33,통관지세관부호_39,통관지세관부호_40,통관지세관부호_41,통관지세관부호_50,통관지세관부호_53,통관지세관부호_56,통관지세관부호_60,통관지세관부호_62,통관지세관부호_70,통관지세관부호_71,통관지세관부호_80,통관지세관부호_81,통관지세관부호_82,통관지세관부호_90,통관지세관부호_100,통관지세관부호_101,통관지세관부호_102,통관지세관부호_110,통관지세관부호_120,통관지세관부호_121,통관지세관부호_122,통관지세관부호_130,...,반입보세구역부호_3_4,반입보세구역부호_3_5,반입보세구역부호_3_6,반입보세구역부호_3_7,반입보세구역부호_3_8,반입보세구역부호_3_9,반입보세구역부호_4_0,반입보세구역부호_4_1,반입보세구역부호_4_2,반입보세구역부호_4_3,반입보세구역부호_4_4,반입보세구역부호_4_5,반입보세구역부호_4_6,반입보세구역부호_4_7,반입보세구역부호_4_9,HS10단위부호_1_1,HS10단위부호_1_2,HS10단위부호_1_3,HS10단위부호_1_4,HS10단위부호_1_5,HS10단위부호_1_6,HS10단위부호_1_7,HS10단위부호_1_8,HS10단위부호_1_9,HS10단위부호_2_0,HS10단위부호_2_1,HS10단위부호_2_2,HS10단위부호_2_3,HS10단위부호_2_4,HS10단위부호_2_5,HS10단위부호_2_6,HS10단위부호_2_7,HS10단위부호_2_8,HS10단위부호_2_9,관세율구분_A,관세율구분_CE,관세율구분_FC,관세율구분_FE,관세율구분_LP,관세율구분_RW
0,0,4.844187,8.593963,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,0,10.303820,13.258812,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,1,10.067242,10.862759,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,0,9.429564,14.388527,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,0,9.661015,15.987683,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
5,0,8.966854,11.516296,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [216]:
df_org_ts.head(6)

,신고번호,신고인부호,신고중량(KG),과세가격원화금액,적출국가코드,원산지국가코드,원산지적출,통관지세관부호_10,통관지세관부호_12,통관지세관부호_13,통관지세관부호_14,통관지세관부호_16,통관지세관부호_17,통관지세관부호_20,통관지세관부호_21,통관지세관부호_23,통관지세관부호_30,통관지세관부호_33,통관지세관부호_39,통관지세관부호_40,통관지세관부호_41,통관지세관부호_50,통관지세관부호_53,통관지세관부호_56,통관지세관부호_60,통관지세관부호_62,통관지세관부호_70,통관지세관부호_71,통관지세관부호_80,통관지세관부호_81,통관지세관부호_82,통관지세관부호_90,통관지세관부호_100,통관지세관부호_101,통관지세관부호_102,통관지세관부호_110,통관지세관부호_120,통관지세관부호_121,통관지세관부호_122,통관지세관부호_130,...,반입보세구역부호_3_4,반입보세구역부호_3_5,반입보세구역부호_3_6,반입보세구역부호_3_7,반입보세구역부호_3_8,반입보세구역부호_3_9,반입보세구역부호_4_0,반입보세구역부호_4_1,반입보세구역부호_4_2,반입보세구역부호_4_3,반입보세구역부호_4_4,반입보세구역부호_4_5,반입보세구역부호_4_6,반입보세구역부호_4_7,반입보세구역부호_4_9,HS10단위부호_1_1,HS10단위부호_1_2,HS10단위부호_1_3,HS10단위부호_1_4,HS10단위부호_1_5,HS10단위부호_1_6,HS10단위부호_1_7,HS10단위부호_1_8,HS10단위부호_1_9,HS10단위부호_2_0,HS10단위부호_2_1,HS10단위부호_2_2,HS10단위부호_2_3,HS10단위부호_2_4,HS10단위부호_2_5,HS10단위부호_2_6,HS10단위부호_2_7,HS10단위부호_2_8,HS10단위부호_2_9,관세율구분_A,관세율구분_CE,관세율구분_FC,관세율구분_FE,관세율구분_LP,관세율구분_RW
0,71490948,0,7.846238,9.868735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,44137011,1,8.296796,13.194446,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,73320079,1,8.297618,13.861786,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0
3,86297956,0,8.268168,9.885334,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0
4,62899313,1,8.512723,11.820565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
5,68507886,0,7.845573,11.322462,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


### 2.3 SMOTE

In [217]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)

df_org_train = df_org.copy()
org_train_y = df_org_train.pop('우범여부')
x,y = smote.fit_sample(df_org_train, org_train_y)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', df_org_train.shape, org_train_y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', x.shape, y.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (76837, 164) (76837,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (118530, 164) (118530,)
SMOTE 적용 후 레이블 값 분포: 
 1    59265
0    59265
dtype: int64


# 3. 모델 생성 및 훈련

### 3.1 모델 import 

In [218]:
# 모델 import
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()

# K-Fold
from sklearn.model_selection import KFold

# Acc, F1 score
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

# warnings
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

### 3.2 K-Fold 기법으로 Split

In [219]:
kf=KFold(n_splits=5, shuffle=True)
test_index_set=set()

# y=df_org.pop('우범여부')
# x=df_org

for train_index, test_index in kf.split(x, y):
    test_index_set.update(test_index)

### 3.3 학습,예측

In [220]:
score = {
    'RF' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'XGBOOST' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'GBM' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'DNN' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
}


In [221]:
def CreateModel(length):
    
    input_shape = (length,)
    mlp_model = models.Sequential()
    mlp_model.add(layers.Dense(units = 10, activation = 'relu', input_shape=input_shape))
    mlp_model.add(layers.Dense(units = 20, activation = 'relu'))
    mlp_model.add(layers.Dense(units = 20, activation = 'relu'))
    mlp_model.add(layers.Dense(units = 2, activation = 'softmax'))
    mlp_model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    return mlp_model

In [222]:
# Model 생성
RF=RandomForestClassifier(max_depth=15, n_estimators=100, random_state=0)
XGBOOST= XGBClassifier(n_estimators=5, max_depth=4,n_jobs=-1)
GBM = GradientBoostingClassifier(max_depth=3, n_estimators=30, random_state=0)
DNN = CreateModel(344)

# 학습, 예측
for train_index, test_index in tqdm(kf.split(x, y)):
    x_train, x_test, y_train, y_test=\
    x[train_index], x[test_index], y[train_index], y[test_index]
    
    RF.fit(x_train, y_train.ravel())
    XGBOOST.fit(x_train, y_train.ravel(), eval_metric=["logloss"], verbose=True)
    GBM.fit(x_train, y_train.ravel())
#     DNN.fit(x_train, encoder.fit_transform(y_train), batch_size=10, epochs=50)
    
    score['RF']['accuracy'].append(RF.score(x_test, y_test.values.ravel()))
    score['RF']['precision'].append(precision_score(RF.predict(x), y))
    score['RF']['recall'].append(recall_score(RF.predict(x), y))
    score['RF']['f1_score'].append(f1_score(RF.predict(x), y))
    
    score['XGBOOST']['accuracy'].append(XGBOOST.score(x_test, y_test.values.ravel()))
    score['XGBOOST']['precision'].append(precision_score(XGBOOST.predict(x), y))
    score['XGBOOST']['recall'].append(recall_score(XGBOOST.predict(x), y))
    score['XGBOOST']['f1_score'].append(f1_score(XGBOOST.predict(x), y))
    
    score['GBM']['accuracy'].append(GBM.score(x_test, y_test.values.ravel()))
    score['GBM']['precision'].append(precision_score(GBM.predict(x), y))
    score['GBM']['recall'].append(recall_score(GBM.predict(x), y))
    score['GBM']['f1_score'].append(f1_score(GBM.predict(x), y))
    
#     score['DNN']['accuracy'].append(DNN.score(x_test, y_test.values.ravel()))
#     score['DNN']['precision'].append(precision_score(DNN.predict(x), y))
#     score['DNN']['recall'].append(recall_score(DNN.predict(x), y))
#     score['DNN']['f1_score'].append(f1_score(DNN.predict(x), y))

    
# Score 출력
print(f'1-1. RandomForestClassifier [평균 Accuracy] :', np.mean(score['RF']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['RF']['accuracy']))
print(f'1-2. RandomForestClassifier [평균 Precision] :', np.mean(score['RF']['precision'])*100 , ', [Precision 편차]:', np.std(score['RF']['precision']))
print(f'1-3. RandomForestClassifier [평균 Recall] :', np.mean(score['RF']['recall'])*100 , ',    [Recall 편차]   :', np.std(score['RF']['recall']))
print(f'1-4. RandomForestClassifier [평균 F1_score] :', np.mean(score['RF']['f1_score'])*100 , ',   [F1_score 편차] :', np.std(score['RF']['f1_score']),'\n')

print(f'2-1. XGBOOSTClassifier [평균 Accuracy] :', np.mean(score['XGBOOST']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['XGBOOST']['accuracy']))
print(f'2-2. XGBOOSTClassifier [평균 Precision] :', np.mean(score['XGBOOST']['precision'])*100 , ', [Precision 편차] :', np.std(score['XGBOOST']['precision']))
print(f'2-3. XGBOOSTClassifier [평균 Recall] :', np.mean(score['XGBOOST']['recall'])*100 , ',   [Recall 편차] :', np.std(score['XGBOOST']['recall']))
print(f'2-4. XGBOOSTClassifier [평균 F1_score] :', np.mean(score['XGBOOST']['f1_score'])*100 , ',  [F1_score 편차] :', np.std(score['XGBOOST']['f1_score']),'\n')

print(f'3-1. GradientBoostingClassifier [평균 Accuracy] :', np.mean(score['GBM']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['GBM']['accuracy']))
print(f'3-2. GradientBoostingClassifier [평균 Precision] :', np.mean(score['GBM']['precision'])*100 , ', [Precision 편차] :', np.std(score['GBM']['precision']))
print(f'3-3. GradientBoostingClassifier [평균 Recall]   :', np.mean(score['GBM']['recall'])*100 , ',    [Recall 편차] :', np.std(score['GBM']['recall']))
print(f'3-4. GradientBoostingClassifier [평균 F1_score] :', np.mean(score['GBM']['f1_score'])*100 , ',   [F1_score 편차] :', np.std(score['GBM']['f1_score']))
                                                                                                                          
# print(f'1-2. DeepNeuralNetworkClassifier [평균 Accuracy] :', np.mean(score['DNN']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['DNN']['accuracy']))
# print(f'1-2. DeepNeuralNetworkClassifier [평균 Precision] :', np.mean(score['DNN']['precision'])*100 , ', [Precision 편차] :', np.std(score['DNN']['precision']))
# print(f'1-2. DeepNeuralNetworkClassifier [평균 Recall]   :', np.mean(score['DNN']['recall'])*100 , ',    [Recall 편차] :', np.std(score['DNN']['recall']))
# print(f'1-2. DeepNeuralNetworkClassifier [평균 F1_score] :', np.mean(score['DNN']['f1_score'])*100 , ',   [F1_score 편차] :', np.std(score['DNN']['f1_score']))




0it [00:00, ?it/s]

AttributeError: ignored